In [1]:
import pandas as pd
import cv2
import numpy as np
import os.path
import matplotlib.pyplot as plt

In [11]:
ROOT = '/Users/shouzeluo/Desktop/Belle/data/imgs/'

file_list = np.asarray([f for f in os.listdir(ROOT) if f.endswith('png')])

In [3]:
img0=cv2.imread(os.path.join(ROOT, 'SJK08100DM1BH6.png'))

H1 = cv2.calcHist([img0], [0], None, [256], [0, 256])
H1=cv2.normalize(H1,H1,0,1,cv2.NORM_MINMAX,-1)

dist = []

for f in file_list:
    img = cv2.imread(os.path.join(ROOT, f))
    tmp = cv2.calcHist([img], [0], None, [256], [0, 256])
    tmp = cv2.normalize(tmp,tmp,0,1,cv2.NORM_MINMAX,-1)
    similarity = cv2.compareHist(H1,tmp,0)
    dist.append(similarity)

dist = np.asarray(dist)
file_name = file_list[dist.argpartition(-2)[-2]]
file_name

'SJK08100DL2BH6.png'

In [16]:


class CompareImage(object):

    def __init__(self, image_1_path, image_2_path):
        self.minimum_commutative_image_diff = 0.25
        self.image_1_path = image_1_path
        self.image_2_path = image_2_path

    def compare_image(self):
        image_1 = cv2.imread(self.image_1_path, 0)
        image_2 = cv2.imread(self.image_2_path, 0)
        img_hist_diff, img_template_diff, commutative_image_diff = self.get_image_difference(image_1, image_2)

#         if img_hist_diff<0.3 and img_template_diff<0.3:
#             if commutative_image_diff < self.minimum_commutative_image_diff:
#                 print("Matched")
#                 return commutative_image_diff
        return commutative_image_diff # random failure value

    @staticmethod
    def get_image_difference(image_1, image_2):
        first_image_hist = cv2.calcHist([image_1], [0], None, [256], [0, 256])
        second_image_hist = cv2.calcHist([image_2], [0], None, [256], [0, 256])

        img_hist_diff = 1-cv2.compareHist(first_image_hist, second_image_hist,0)
        img_template_probability_match = cv2.matchTemplate(first_image_hist, second_image_hist, cv2.TM_CCOEFF_NORMED)[0][0]
        img_template_diff = 1 - img_template_probability_match

        # taking only 10% of histogram diff, since it's less accurate than template method
        commutative_image_diff = (img_hist_diff / 10) + img_template_diff
        return [img_hist_diff,img_template_diff,commutative_image_diff]



    

In [21]:
p1 = os.path.join(ROOT, 'SJK01500DQ1AM6.png')
dist = []
for f in file_list:
    p2 = os.path.join(ROOT, f)

    compare_image = CompareImage(p1, p2)
    image_difference = compare_image.compare_image()
    dist.append(image_difference)

In [22]:
dist = np.asarray(dist)
file_list[dist.argsort()]

array(['SJK01500DQ1AM6.png', 'SJK11600DM1BK6.png', 'SJK01500DB1AM6.png',
       'SJK08100DP2BH6.png', 'SJK11600DP1BK6.png', 'SJK08100DB1BH6.png',
       'S79E5101DM1AM6.png', 'SJK08100DM2BH6.png', 'SC5C9101DL1AM6.png',
       'S79E5101DL5AM6.png', 'S79E5101DM5AM6.png', 'SJK08100DL2BH6.png',
       'SJK18400DK1BH6.png', 'SJK08100DM1BH6.png', 'SC5C9101DU1AM6.png',
       'SJK18400DM1BH6.png', 'SJK08100DQ1BH6.png', 'SC5C9101DP2AM6.png'],
      dtype='|S18')